## Inflation data

Get the month on month inflation data from stat.gov.pl




In [42]:
from requests import get
from csv import reader
from datetime import date
import pandas as pd


In [43]:
data, final_data = {}, {}
url = 'https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultstronaopisowa/4741/1/1/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.csv'
content = get(url).content.decode('windows-1250').replace(',', '.').replace(';', ',')
lines = list(reader(content.split('\n')))[1:-1]
for record in lines:
    _, _, method, year, month, value, _, _, _ = record
    if method != 'Poprzedni miesiąc = 100': continue
    record_date = date(int(year), int(month), 1)
    if value == '': continue
    data[record_date] = float(value)

data
df = pd.DataFrame(data.items(), columns=['Date', 'Value'])

df.dropna(inplace=True)
df.sort_values(by='Date', ascending=True, inplace=True)
df.set_index('Date', inplace=True)

df['inter'] = round((df['Value']/100)-1, 3)

df['last_12m'] = (df['inter']).rolling(window=12).sum() * 100 + 100
df.sort_values(by='Date', ascending=False, inplace=True)
df.head(40)


,Value,inter,last_12m
Date,,,
2023-09-01,99.6,-0.004,107.5
2023-08-01,100.0,0.000,109.5
2023-07-01,99.8,-0.002,110.3
2023-06-01,100.0,0.000,111.0
2023-05-01,100.0,0.000,112.5
2023-04-01,100.7,0.007,114.2
2023-03-01,101.1,0.011,115.5
2023-02-01,101.2,0.012,117.7
2023-01-01,102.5,0.025,116.2


## Inflation data

Get the month on month inflation data from stat.gov.pl




In [44]:
# scatter plot with index date and value

import plotly.express as ex

fig = ex.scatter(df, x=df.index, y='Value', title='CPI', labels={'x':'Date', 'y':'Value'})
fig.add_scatter(x=df.index, y=df['last_12m'], mode='lines', name='Last 12 months')
fig.show()




## save the data to a csv file

In [48]:
# save data from 2010 to now

df['year'] = pd.DatetimeIndex(df.index).year
df
(df
 .query('year >= 2006')
 .sort_index(ascending=True)
 [['inter']]
 .rename(columns={'inter': 'value'})
 .rename_axis('date')
 .to_csv('../data/poland_monthly_cpi.csv')
 )

## Predict the inflation rate based on the mean and std deviation of the last 20 years

In [55]:
base_data = (
    df
    .query('year >= 2000')
    .sort_index(ascending=True)
    [['inter']]
    .rename(columns={'inter': 'value'})
    .rename_axis('date')
)
mean_inflation = base_data['value'].mean()
std_deviation = base_data['value'].std()


In [67]:
import numpy as np

months_to_simulate = 20*12
simulated_inflations = np.random.normal(mean_inflation, std_deviation, months_to_simulate)


array([ 1.87064428e-03, -2.21393801e-04,  1.93587047e-03, -3.22141299e-04,
       -5.60647634e-03,  1.95811413e-03, -1.86536303e-03,  3.84562016e-04,
        9.59035076e-03,  7.44780715e-03,  1.24182880e-02, -1.53342900e-03,
       -4.76687742e-03,  3.77077334e-05,  9.06741304e-04, -1.59685070e-03,
        5.69322991e-03,  5.54606181e-03, -4.20284851e-03,  5.49273667e-03,
        5.39164275e-03,  3.34937881e-03, -4.34742950e-04,  9.85489011e-03,
        2.04574764e-03,  6.54360977e-03,  1.11545283e-02, -1.23998039e-02,
       -1.77041224e-03,  6.78314481e-03,  6.66604772e-03,  6.25710073e-03,
        5.27684644e-03,  5.11962114e-04,  1.67447834e-03,  1.01582229e-02,
        8.26259427e-03, -3.29267918e-05,  2.88182764e-03,  3.22806255e-03,
        3.69250688e-03, -2.95000773e-03,  1.34232986e-03,  6.28833447e-03,
        1.34219945e-02,  9.72288654e-03,  6.74292514e-03,  5.39246473e-03,
        6.74205002e-03,  2.51379993e-03,  1.20043737e-02,  8.30100541e-03,
        8.91759654e-03, -

In [68]:
last_date = base_data.index[-1]
dates_future = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=months_to_simulate, freq='ME')

simulated_data = pd.DataFrame({'Date': dates_future, 'value': simulated_inflations})
simulated_data.set_index('Date', inplace=True)
simulated_data

simulated_data['last_12m'] = (simulated_data['value']).rolling(window=12).sum()

fig = ex.scatter(simulated_data, x=simulated_data.index, y='value', title='Simulated Inflation', labels={'x':'Date', 'y':'Value'})
fig.add_scatter(x=simulated_data.index, y=simulated_data['last_12m'], mode='lines', name='Last 12 months')
fig.show()

## save simulated inflation data to csv

In [70]:
(simulated_data
 [['value']]
 .rename_axis('date')
 .to_csv('../data/monthly_cpi_simulated_PLN.csv')
)